In [7]:
import math
import numpy
import scipy

In [8]:
def black_scholes(spot_price, strike_price, risk_rate, mature_time, volitility, callput):
    if callput == "call":
        return numpy.round(((spot_price * scipy.stats.norm.cdf(((math.log(spot_price / strike_price) + (
                risk_rate + ((volitility ** 2) / 2)) * mature_time) / ((
                volitility * math.sqrt(mature_time)))))) - strike_price * math.exp(
            -risk_rate * mature_time) * scipy.stats.norm.cdf((((math.log(spot_price / strike_price) + (
                risk_rate + ((volitility ** 2) / 2)) * mature_time) / ((volitility * math.sqrt(mature_time)))) - (
                                                                      volitility * math.sqrt(mature_time))))),
                           decimals=1)
    else:
        return numpy.round((strike_price * math.exp(-risk_rate * mature_time) * scipy.stats.norm.cdf(
            -(((math.log(spot_price / strike_price) + (risk_rate + ((volitility ** 2) / 2)) * mature_time) / (
                (volitility * math.sqrt(mature_time)))) - (
                      volitility * math.sqrt(mature_time)))) - spot_price * scipy.stats.norm.cdf(-(
                (math.log(spot_price / strike_price) + (risk_rate + ((volitility ** 2) / 2)) * mature_time) / (
            (volitility * math.sqrt(mature_time)))))), decimals=1)

In [9]:
def binomial_tree(stock_price, strike_price, interest_rate, volatility, time, steps, option_type, exercise_type):
    stock_tree = [[stock_price]]
    for step in range(1, steps + 1):
        price_row = []
        for item in range(0, step + 1):
            if item == step:
                price = stock_tree[step - 1][step - 1] * (math.exp(volatility * numpy.sqrt((time / steps))))
            elif item == 0:
                price = stock_tree[step - 1][0] * (1 / (math.exp(volatility * numpy.sqrt((time / steps)))))
            else:
                price = stock_tree[step - 1][item - 1] * (math.exp(volatility * numpy.sqrt((time / steps))))
            price_row.append(price)
        stock_tree.append(price_row)
    tree = [[-1]]
    for step in range(1, steps):
        child = []
        for item in range(step + 1):
            child.append(-1)
        tree.append(child)
    level = []
    for step in range(steps + 1):
        price = stock_tree[steps][step]
        if option_type == 'call':
            if price <= strike_price:
                value = 0
            else:
                value = price - strike_price
        else:
            if strike_price > price:
                value = strike_price - price
            else:
                value = 0
        level.append(value)
    tree.append(level)
    for i in range(steps - 1, -1, -1):
        prev_row = tree[i + 1]
        for j in range(i + 1):
            if exercise_type == 'american' and option_type == 'put':
                american_value = max(value, strike_price - stock_tree[i][j])
                item_value = (math.exp(-interest_rate * (time / steps))) * ((math.e ** (
                        interest_rate * (time / steps)) - (1 / math.e ** (
                        volatility * math.sqrt((time / steps))))) / ((math.e ** (
                        volatility * math.sqrt((time / steps)))) - (1 / math.e ** (
                        volatility * math.sqrt((time / steps))))) * prev_row[j + 1] + (1 - (
                        math.exp(interest_rate * (time / steps)) - (
                        1 / math.exp(volatility * math.sqrt((time / steps))))) / ((math.exp(
                    volatility * numpy.sqrt((time / steps)))) - (1 / math.e ** (
                        volatility * numpy.sqrt((time / steps)))))) * prev_row[j])
                tree[i][j] = max(american_value, item_value)
            else:
                tree[i][j] = (math.exp(-interest_rate * (time / steps))) * ((math.e ** (
                        interest_rate * (time / steps)) - (1 / math.e ** (
                        volatility * math.sqrt((time / steps))))) / ((math.e ** (
                        volatility * math.sqrt((time / steps)))) - (1 / math.e ** (
                        volatility * math.sqrt((time / steps))))) * prev_row[j + 1] + (1 - (
                        math.e ** (interest_rate * (time / steps)) - (
                        1 / math.e ** (volatility * math.sqrt((time / steps))))) / ((math.e ** (
                        volatility * math.sqrt((time / steps)))) - (1 / math.e ** (
                        volatility * math.sqrt((time / steps)))))) * prev_row[j])
    tree = tree[0][0]
    return numpy.round(tree, decimals=1)

In [10]:
def monte_carlo(stock_price, strike_price, time, interest_rate, volatility, type, steps, simulations):
    st = stock_price * numpy.exp(numpy.cumsum(((interest_rate - 0.5 * volatility * volatility) * time / steps) + (
            volatility * numpy.sqrt(time / steps)) * numpy.random.standard_normal(size=(steps, simulations)), axis=0))
    if type == "put":
        values = numpy.maximum(strike_price - st[-1], 0)
    else:
        values = numpy.maximum(st[-1] - strike_price, 0)
    val = 0
    for i in range(0, len(values)):
        val = values[i] + val
    return numpy.round(val / len(values) * math.exp(-interest_rate * time), decimals=1)

In [11]:
print("black_scholes call x1", black_scholes(100, 120, 0.05, 0.5, 0.2, "call"))
print("black_scholes put x1", black_scholes(100, 120, 0.05, 0.5, 0.2, "put"))

print("black_scholes call x2", black_scholes(100, 98, 0.05, 0.5, 0.2, "call"))
print("black_scholes put x2", black_scholes(100, 98, 0.05, 0.5, 0.2, "put"))

print("binomial tree call euro x1", binomial_tree(100, 120, 0.05, 0.20, 0.5, 180, 'call', 'europiean'))
print("binomial tree put euro x1", binomial_tree(100, 120, 0.05, 0.20, 0.5, 180, 'put', 'europiean'))
print("binomial tree call amer x1", binomial_tree(100, 120, 0.05, 0.20, 0.5, 180, 'call', 'american'))
print("binomial tree put amer x1", binomial_tree(100, 120, 0.05, 0.20, 0.5, 180, 'put', 'american'))

print("binomial tree call euro x2", binomial_tree(100, 98, 0.05, 0.20, 0.5, 180, 'call', 'europiean'))
print("binomial tree put euro x2", binomial_tree(100, 98, 0.05, 0.20, 0.5, 180, 'put', 'europiean'))
print("binomial tree call amer x2", binomial_tree(100, 98, 0.05, 0.20, 0.5, 180, 'call', 'american'))
print("binomial tree put amer x2", binomial_tree(100, 98, 0.05, 0.20, 0.5, 180, 'put', 'american'))

print("monte carlo call x1", monte_carlo(100, 120, 0.5, 0.05, 0.2, "call", 500, 200000))
print("monte carlo put x1", monte_carlo(100, 120, 0.5, 0.05, 0.2, "put", 500, 200000))

print("monte carlo call x2", monte_carlo(100, 98, 0.5, 0.05, 0.2, "call", 500, 200000))
print("monte carlo put x2", monte_carlo(100, 98, 0.5, 0.05, 0.2, "put", 500, 200000))

black_scholes call x1 1.0
black_scholes put x1 18.1
black_scholes call x2 8.0
black_scholes put x2 3.6
binomial tree call euro x1 1.0
binomial tree put euro x1 18.1
binomial tree call amer x1 1.0
binomial tree put amer x1 20
binomial tree call euro x2 8.0
binomial tree put euro x2 3.6
binomial tree call amer x2 8.0
binomial tree put amer x2 3.8
monte carlo call x1 1.0
monte carlo put x1 18.1
monte carlo call x2 8.0
monte carlo put x2 3.6
